![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [128]:
import pandas as pd
import numpy as np

#Step 1 - Reading in and splitting the data
#Load the csv file and create three DataFrames
#Read in the dataset as a pandas DataFrame
data = pd.read_csv("bank_marketing.csv")

#Split the data by subsetting DataFrame
client = data[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]
campaign = data[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome", "month", "day"]]
economics = data[["client_id", "cons_price_idx", "euribor_three_months"]]

In [129]:
#Check data type
client.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default    object
mortgage          object
dtype: object

In [130]:
#Explore dataset
print(client.head())

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic.4y             no       no
1          1   57   services  married  high.school        unknown       no
2          2   37   services  married  high.school             no      yes
3          3   40     admin.  married     basic.6y             no       no
4          4   56   services  married  high.school             no       no


In [131]:
#Explore dataset
client.credit_default.unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [132]:
#Check data types
campaign.dtypes

client_id                      int64
number_contacts                int64
contact_duration               int64
previous_campaign_contacts     int64
previous_outcome              object
campaign_outcome              object
month                         object
day                            int64
dtype: object

In [133]:
#Explore dataset
campaign.previous_outcome.unique()

array(['nonexistent', 'failure', 'success'], dtype=object)

In [134]:
#Explore dataset
campaign.campaign_outcome.unique()

array(['no', 'yes'], dtype=object)

In [135]:
#Explore dataset
print(campaign.head())

   client_id  number_contacts  contact_duration  ...  campaign_outcome month day
0          0                1               261  ...                no   may  13
1          1                1               149  ...                no   may  19
2          2                1               226  ...                no   may  23
3          3                1               151  ...                no   may  27
4          4                1               307  ...                no   may   3

[5 rows x 8 columns]


In [136]:
#Check data types
economics.dtypes

client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object

In [137]:
#Cleaning the data - clean, create, and delete columns
#Clean client columns
client["job"] = client["job"].replace(".", "_")
client["education"] = client["education"].replace(".", "_")
client["education"] = client["education"].replace("uknown", np.NaN)

In [138]:
#Create new columns - campaign
#Create year column
campaign["year"] = 2022

print(campaign.head())

   client_id  number_contacts  contact_duration  ...  month day  year
0          0                1               261  ...    may  13  2022
1          1                1               149  ...    may  19  2022
2          2                1               226  ...    may  23  2022
3          3                1               151  ...    may  27  2022
4          4                1               307  ...    may   3  2022

[5 rows x 9 columns]


In [139]:
#Create last contact day column
campaign["date"] = campaign["month"].astype(str) + " " + campaign["day"].astype(str) + " " + campaign["year"].astype(str)

In [140]:
print(campaign.head())

   client_id  number_contacts  contact_duration  ...  day  year         date
0          0                1               261  ...   13  2022  may 13 2022
1          1                1               149  ...   19  2022  may 19 2022
2          2                1               226  ...   23  2022  may 23 2022
3          3                1               151  ...   27  2022  may 27 2022
4          4                1               307  ...    3  2022   may 3 2022

[5 rows x 10 columns]


In [141]:
campaign["last_contact_date"] = pd.to_datetime(campaign["date"])

In [142]:
print(campaign.head())

   client_id  number_contacts  ...         date  last_contact_date
0          0                1  ...  may 13 2022         2022-05-13
1          1                1  ...  may 19 2022         2022-05-19
2          2                1  ...  may 23 2022         2022-05-23
3          3                1  ...  may 27 2022         2022-05-27
4          4                1  ...   may 3 2022         2022-05-03

[5 rows x 11 columns]


In [143]:
#Delete year, month, day, date columns
campaign = campaign.drop(["month", "day", "year", "date"], axis=1)

In [144]:
campaign.dtypes

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                      object
campaign_outcome                      object
last_contact_date             datetime64[ns]
dtype: object

In [145]:
#Converting data types - client
client["credit_default"] = client["credit_default"].map({"yes": 1, "no": 0, "unknown": 0})
client["credit_default"] = client["credit_default"].astype(bool)
client["mortgage"] = client["mortgage"].astype(bool)

In [146]:
#Converting data types - campaign
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "failure": 0, "nonexistent": 0})
campaign["previous_outcome"] = campaign["previous_outcome"].astype(bool)

campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})
campaign["campaign_outcome"] = campaign["campaign_outcome"].astype(bool)

In [147]:
print(campaign.head())

   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1  ...             False         2022-05-23
3          3                1  ...             False         2022-05-27
4          4                1  ...             False         2022-05-03

[5 rows x 7 columns]


In [148]:
#Saving the data

#Save the three DataFrames as csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)